In [1]:
import pandas as pd
import numpy as np
import scanpy.api as sc
import anndata
import os
import subprocess

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

scanpy==1.4 anndata==0.6.18 numpy==1.15.4 scipy==1.2.0 pandas==0.24.1 scikit-learn==0.20.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [2]:
train_path = "../data/pancreas.h5ad"
if os.path.isfile(train_path):
    adata = sc.read(train_path)
else:
    train_url = "https://www.dropbox.com/s/qj1jlm9w10wmt0u/pancreas.h5ad?dl=1"
    t_dl = wget.download(train_url, train_path)
    adata = sc.read(train_path)
adata = anndata.AnnData(X=np.expm1(adata.raw.X), var=adata.raw.var, obs=adata.obs)
sc.pp.filter_cells(adata, min_genes=600)

filtered out 376 cells that have less than 600 genes expressed


Attempts to run Scanorama within a Jupyter Notebook have been unsuccessful. However, exporting the data outside of the notebook and running Scanorama's entire workflow, including turning the data into `.npz` pickles results in functional output. So let's export the rawest form of the count matrices for Scanorama to process.

This notebook won't run immediately on your system as you need a Scanorama GitHub clone.

In [3]:
df1 = pd.DataFrame(data=adata[adata.obs['sample']=='Baron'].X.todense().transpose(),
                  index=adata[adata.obs['sample']=='Baron'].var_names,
                  columns=np.arange(np.sum(adata.obs['sample']=='Baron')))

df2 = pd.DataFrame(data=adata[adata.obs['sample']=='Muraro'].X.todense().transpose(),
                  index=adata[adata.obs['sample']=='Muraro'].var_names,
                  columns=np.arange(np.sum(adata.obs['sample']=='Muraro')))

df3 = pd.DataFrame(data=adata[adata.obs['sample']=='Segerstolpe'].X.todense().transpose(),
                  index=adata[adata.obs['sample']=='Segerstolpe'].var_names,
                  columns=np.arange(np.sum(adata.obs['sample']=='Segerstolpe')))

df4 = pd.DataFrame(data=adata[adata.obs['sample']=='Wang'].X.todense().transpose(),
                  index=adata[adata.obs['sample']=='Wang'].var_names,
                  columns=np.arange(np.sum(adata.obs['sample']=='Wang')))

if not os.path.exists('../../scanorama/data/4panc'):
    os.makedirs('../../scanorama/data/4panc')
df1.to_csv('../../scanorama/data/4panc/baron.txt',sep='\t')
df2.to_csv('../../scanorama/data/4panc/muraro.txt',sep='\t')
df3.to_csv('../../scanorama/data/4panc/segerstolpe.txt',sep='\t')
df4.to_csv('../../scanorama/data/4panc/wang.txt',sep='\t')

Run Scanorama. To do this, you need to create this file at `conf/4panc.txt`:

	data/4panc/baron
	data/4panc/muraro
	data/4panc/segerstolpe
	data/4panc/wang

...and this file at `bin/4panc.py`:

    import numpy as np
    from sklearn.preprocessing import normalize, LabelEncoder
    import sys

    from process import load_names, merge_datasets, save_datasets
    from scanorama import correct, visualize, process_data
    from scanorama import dimensionality_reduce

    import time
    from datetime import timedelta

    data_names = [
        'data/4panc/baron',
        'data/4panc/muraro',
        'data/4panc/segerstolpe',
        'data/4panc/wang'
    ]

    if __name__ == '__main__':
        datasets, genes_list, n_cells = load_names(data_names)
        t1 = time.time()
        datasets, genes = correct(datasets, genes_list)
        datasets = [ normalize(ds, axis=1) for ds in datasets ]
        t2 = time.time()
        print('Took '+str(timedelta(seconds=t2-t1)))

        save_datasets(datasets, genes, data_names)

As you can see, we have timing baked into the script, only measuring the time required to perform the actual batch correction. We export the resulting expression profiles so that we can analyse them within the notebook.

In [8]:
os.chdir('../../scanorama/')

subprocess.run('python bin/process.py conf/4panc.txt', shell=True)
res = subprocess.run('python bin/4panc.py', shell=True, stdout=subprocess.PIPE)
print(res.stdout.decode('utf-8').split('\n'))

os.chdir('../bbknn/examples/')

['Loaded data/4panc/baron with 24516 genes and 8569 cells', 'Loaded data/4panc/muraro with 24516 genes and 2126 cells', 'Loaded data/4panc/segerstolpe with 24516 genes and 2987 cells', 'Loaded data/4panc/wang with 24516 genes and 635 cells', 'Found 14317 cells among all datasets', 'Found 24516 genes among all datasets', '[[0.         0.12699906 0.37261466 0.08503937]', ' [0.         0.         0.38523048 0.0976378 ]', ' [0.         0.         0.         0.82047244]', ' [0.         0.         0.         0.        ]]', 'Processing datasets (2, 3)', 'Processing datasets (1, 2)', 'Processing datasets (0, 2)', 'Processing datasets (0, 1)', 'Took 0:02:27.044554', '']


The last line in the standard output dump above captures a run time of two minutes. Now that that's done, import the expression back into the notebook and make a new object.

In [9]:
sc1 = pd.read_table('../../scanorama/data/4panc/baron.scanorama_corrected.txt',index_col=0)
sc2 = pd.read_table('../../scanorama/data/4panc/muraro.scanorama_corrected.txt',index_col=0)
sc3 = pd.read_table('../../scanorama/data/4panc/segerstolpe.scanorama_corrected.txt',index_col=0)
sc4 = pd.read_table('../../scanorama/data/4panc/wang.scanorama_corrected.txt',index_col=0)

adata_scanorama = anndata.AnnData(X=np.vstack((sc1.values.transpose(),sc2.values.transpose(),
                                               sc3.values.transpose(),sc4.values.transpose())),
                                  obs=adata.obs, var=sc1.index)

/anaconda3/envs/scgen/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/anaconda3/envs/scgen/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/anaconda3/envs/scgen/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/scgen/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


The final object export.

In [12]:
adata_scanorama.write('objects-pancreas/pancreas_scanorama.h5ad')